In [12]:
import numpy as np
import time
import itertools
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import fetch_openml 
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler # 스케일러 추가
from scipy.special import expit as sigmoid, logit
from typing import List, Tuple, Optional, Dict
import sys

# ==============================================================================
# PART 0: CORE FRAMEWORK FUNCTIONS (Self-Contained)
# (SigmoidWarping, CompressionManagerND, hill_climb_with_compression_nd, etc.)
# ==============================================================================

# --- Sigmoid Warping and Compression Metadata Classes ---
# (Classes remain unchanged)

class SigmoidWarping:
    def __init__(self, node_index, length, steepness=5.0):
        self.node_start = int(node_index)
        self.length = int(length)
        self.node_end = self.node_start + self.length
        self.steepness = float(steepness)
        self.shift = self.length - 1 

    def forward(self, node):
        node = np.atleast_1d(node).astype(float)
        position = np.zeros(len(node), dtype=float)
        for i, n in enumerate(node):
            if n < self.node_start: position[i] = float(n)
            elif n > self.node_end: position[i] = float(n) - self.shift
            else:
                if n == self.node_start: position[i] = float(self.node_start)
                elif n == self.node_end: position[i] = float(self.node_start + 1)
                else:
                    t = (n - self.node_start) / self.length
                    x = self.steepness * (t - 0.5)
                    position[i] = self.node_start + sigmoid(x)
        return position[0] if len(node) == 1 else position

    def inverse(self, position):
        position = np.atleast_1d(position).astype(float)
        node = np.zeros(len(position), dtype=int)
        for i, pos in enumerate(position):
            if pos < self.node_start: node[i] = int(np.floor(pos))
            elif pos >= self.node_start + 1.0: node[i] = int(np.ceil(pos + self.shift))
            else:
                s = np.clip(pos - self.node_start, 0.0, 1.0)
                if s <= 0.01: node[i] = self.node_start
                elif s >= 0.99: node[i] = self.node_end
                else:
                    x = logit(s)
                    t = x / self.steepness + 0.5
                    node_f = self.node_start + t * self.length
                    node[i] = int(round(node_f))
        return node[0] if len(node) == 1 else node

class MetadataCompressionOriginalSpace:
    def __init__(self, compressions_x_space=None, steepness=5.0):
        self.metadata_x = sorted(compressions_x_space or [], key=lambda x: x[0])
        self.steepness = float(steepness)
        self.warpings = []
        if self.metadata_x: self._build_warpings()

    def _build_warpings(self):
        cumulative_shift = 0
        for i, (x_start, x_length) in enumerate(self.metadata_x):
            z_start = x_start - cumulative_shift
            z_length = x_length
            warping = SigmoidWarping(z_start, z_length, self.steepness)
            self.warpings.append(warping)
            cumulative_shift += (z_length - 1)

    def forward(self, node):
        position = node
        for warping in self.warpings: position = warping.forward(position)
        return position

    def inverse(self, position):
        node = position
        for warping in reversed(self.warpings): node = warping.inverse(node)
        return node

def detect_compression_basin(fitness_func, local_min_x, max_search=100):
    local_min_fitness = fitness_func(local_min_x)
    
    # LEFT search logic...
    left_boundary = local_min_x
    farthest_left_equal = local_min_x
    found_left_equal = False
    found_left_exit = False
    for i in range(1, max_search + 1):
        current_x = local_min_x - i
        current_fitness = fitness_func(current_x)
        if abs(current_fitness - local_min_fitness) < 1e-9: farthest_left_equal = current_x; found_left_equal = True
        elif current_fitness > local_min_fitness: pass
        else: left_boundary = current_x + 1; found_left_exit = True; break
    else: left_boundary = local_min_x - max_search
    if not found_left_exit and found_left_equal: left_boundary = farthest_left_equal

    # RIGHT search logic...
    right_boundary = local_min_x
    farthest_right_equal = local_min_x
    found_right_equal = False
    found_right_exit = False
    for i in range(1, max_search + 1):
        current_x = local_min_x + i
        current_fitness = fitness_func(current_x)
        if abs(current_fitness - local_min_fitness) < 1e-9: farthest_right_equal = current_x; found_right_equal = True
        elif current_fitness > local_min_fitness: pass
        else: right_boundary = current_x - 1; found_right_exit = True; break
    else: right_boundary = local_min_x + max_search
    if not found_right_exit and found_right_equal: right_boundary = farthest_right_equal

    basin_length = right_boundary - left_boundary + 1
    if basin_length < 2: return None
    if not (found_left_equal or found_right_equal or found_left_exit or found_right_exit): return None
    return (left_boundary, basin_length)

def merge_overlapping_compressions(compressions):
    if not compressions: return []
    sorted_comps = sorted(compressions, key=lambda x: x[0])
    merged = []
    for start, length in sorted_comps:
        end = start + length - 1
        if not merged: merged.append((start, length)); continue
        last_start, last_length = merged[-1]
        last_end = last_start + last_length - 1
        if start <= last_end + 1:
            new_start = min(start, last_start)
            new_end = max(end, last_end)
            new_length = new_end - new_start + 1
            merged[-1] = (new_start, new_length)
        else: merged.append((start, length))
    return merged

class CompressionManagerND:
    def __init__(self, dim, steepness=5.0):
        self.dim = dim
        self.steepness = float(steepness)
        self.dim_compressions = [{} for _ in range(dim)]
        self.dim_systems = [{} for _ in range(dim)]
    
    def update_dimension(self, vary_dim, fixed_coords, basin):
        if basin is None: return
        comps = self.dim_compressions[vary_dim].get(fixed_coords, [])
        comps.append(basin)
        comps = merge_overlapping_compressions(comps)
        self.dim_compressions[vary_dim][fixed_coords] = comps
        self.dim_systems[vary_dim][fixed_coords] = MetadataCompressionOriginalSpace(comps, self.steepness)
    
    def get_system(self, vary_dim, fixed_coords):
        return self.dim_systems[vary_dim].get(fixed_coords, None)

def detect_basin_along_dimension(fitness_func_nd, point, vary_dim, max_search=100):
    def f1d(val):
        new_point = list(point)
        new_point[vary_dim] = int(val)
        return fitness_func_nd(tuple(new_point))
    
    basin = detect_compression_basin(f1d, local_min_x=int(point[vary_dim]), max_search=max_search)
    
    min_val = int(point[vary_dim])
    min_f = fitness_func_nd(point)
    
    if basin:
        start, length = basin
        print(f"    [Dim {vary_dim}] (1D Detect @ x={min_val}, f={min_f:.4f}) -> Basin: [{start}, {start+length-1}] (len={length})")
    else:
        print(f"    [Dim {vary_dim}] (1D Detect @ x={min_val}, f={min_f:.4f}) -> No compressible basin found.")
        
    return basin

def hill_climb_with_compression_nd(fitness_func_nd, start_point, dim, max_iterations=10, basin_max_search=100, global_min_threshold=1e-6):
    traj = []
    cm = CompressionManagerND(dim, steepness=5.0)
    point = tuple(int(x) for x in start_point)
    f = fitness_func_nd(point)
    traj.append((point, f, False))
    
    print(f"\n🚀 {dim}D COMPRESSION climb start at {point}, f={f:.4f}\n")

    for it in range(max_iterations):
        print(f"================================================================================")
        print(f"🔄 Iteration {it+1}/{max_iterations}")
        print(f"================================================================================")
        
        step_count = 0
        while True:
            candidates = []
            # 1. O(D) Axis-aligned Neighbors
            for d in range(dim):
                fixed_coords = tuple(point[i] for i in range(dim) if i != d)
                comp_sys = cm.get_system(d, fixed_coords)
                if comp_sys is not None:
                    z = comp_sys.forward(point[d])
                    nm, np_ = comp_sys.inverse(z - 1), comp_sys.inverse(z + 1)
                else:
                    nm, np_ = point[d] - 1, point[d] + 1
                
                pm = list(point); pm[d] = nm
                pp = list(point); pp[d] = np_
                candidates.append((tuple(pm), fitness_func_nd(tuple(pm))))
                candidates.append((tuple(pp), fitness_func_nd(tuple(pp))))
            
            # 2. O(D^2) Diagonal Neighbors
            for d1, d2 in itertools.combinations(range(dim), 2):
                for o1 in [1, -1]:
                    for o2 in [1, -1]:
                        np_ = list(point); np_[d1] += o1; np_[d2] += o2
                        candidates.append((tuple(np_), fitness_func_nd(tuple(np_))))
            
            best_point, best_f = point, f
            for cp, cf in candidates:
                if cf < best_f - 1e-9: best_point, best_f = cp, cf
            
            if best_f < f - 1e-9:
                point, f = best_point, best_f
                used_comp = any(cm.get_system(d, tuple(point[i] for i in range(dim) if i != d)) is not None for d in range(dim))
                traj.append((point, f, used_comp))
                step_count += 1
            else:
                print(f"  📍 Climbed {step_count} steps, now at {point}, f={f:.6g}")
                break
        
        if abs(f) < global_min_threshold:
            print("\n🎉 SUCCESS: reached near-global minimum")
            break
        
        # Basin Detection
        print(f"\n⚠️ STUCK at local minimum {point}, f={f:.6g}")
        print(f"  🔍 Detecting basins along all {dim} dimensions...")
        basins = {}
        for d in range(dim):
            basin = detect_basin_along_dimension(fitness_func_nd, point, d, max_search=basin_max_search)
            if basin:
                fixed_coords = tuple(point[i] for i in range(dim) if i != d)
                cm.update_dimension(d, fixed_coords, basin)
                basins[d] = basin
        
        # Restart Logic
        restart_candidates = []
        for d, (b_start, b_len) in basins.items():
            b_end = b_start + b_len - 1
            rp1 = list(point); rp1[d] = b_start - 1
            rp2 = list(point); rp2[d] = b_end + 1
            restart_candidates.append((tuple(rp1), fitness_func_nd(tuple(rp1))))
            restart_candidates.append((tuple(rp2), fitness_func_nd(tuple(rp2))))
        
        if restart_candidates:
            best_r_p, best_r_f = min(restart_candidates, key=lambda t: t[1])
            if best_r_f < f - 1e-9:
                print(f"  ➡️ Restarting from {best_r_p}, f={best_r_f:.4f}")
                point, f = best_r_p, best_r_f
                traj.append((point, f, True))
            else:
                print("\n  ⚠️ Restart candidates didn't improve fitness. Stopping.")
                break
        else:
            break
            
    print(f"\n🏁 FINAL {dim}D COMPRESSION RESULTS 🏁")
    print(f"  Final position: {point}, Final fitness: {f:.6g}, Total steps: {len(traj)}")
    return traj

def hill_climb_simple_nd(fitness_func_nd, start_point, dim, max_steps=2000):
    point = tuple(int(x) for x in start_point)
    f = fitness_func_nd(point)
    traj = [(point, f)]
    print(f"\n🚀 {dim}D BASELINE climb start at {point}, f={f:.4f}\n")

    for _ in range(max_steps):
        candidates = []
        # 1. O(D)
        for d in range(dim):
            nm = list(point); nm[d] -= 1
            np_ = list(point); np_[d] += 1
            candidates.append((tuple(nm), fitness_func_nd(tuple(nm))))
            candidates.append((tuple(np_), fitness_func_nd(tuple(np_))))
        # 2. O(D^2) Diagonal
        for d1, d2 in itertools.combinations(range(dim), 2):
            for o1 in [1, -1]:
                for o2 in [1, -1]:
                    np_ = list(point); np_[d1] += o1; np_[d2] += o2
                    candidates.append((tuple(np_), fitness_func_nd(tuple(np_))))

        best_point, best_f = point, f
        for cp, cf in candidates:
            if cf < best_f - 1e-9: best_point, best_f = cp, cf
        
        if best_f < f - 1e-9:
            point, f = best_point, best_f
            traj.append((point, f))
        else:
            break
            
    print(f"🏁 FINAL {dim}D BASELINE RESULTS 🏁")
    print(f"  Final position: {point}, Final fitness: {f:.6g}, Total steps: {len(traj)}")
    return traj

# ==============================================================================
# 3. RANDOM FOREST HPO IMPLEMENTATION
# ==============================================================================

# Global Data Cache for speed
X_TRAIN, Y_TRAIN, X_VAL, Y_VAL = None, None, None, None

def load_fashion_mnist_for_hpo(n_samples: int = 5000, random_state: int = 42) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Fashion MNIST를 로드하고 Subsample 및 스케일링을 수행합니다. (데이터 검증 로깅 추가)
    """
    global X_TRAIN, Y_TRAIN, X_VAL, Y_VAL
    
    if X_TRAIN is not None:
        return X_TRAIN, Y_TRAIN, X_VAL, Y_VAL

    print("Loading Fashion MNIST (Subsample for increased difficulty)...")
    
    try:
        mnist = fetch_openml('Fashion-MNIST', version=1, cache=True, as_frame=False)
        X, y = mnist.data, mnist.target
        
        # 🚨 [수정된 부분]: 타겟 y를 np.int64로 명확하게 변환
        X = X.astype(np.float64) 
        # y 데이터를 안전하게 정수형으로 변환 (문자열 '0', '1' 등도 처리 가능)
        y = y.astype(np.int64)
        
        # 🚨 [검증 1] 로드 직후 데이터 타입 및 형태 확인
        print(f"  [CHECK 1: Raw Data] X Shape: {X.shape}, Y Type: {y.dtype}")
        print(f"  [CHECK 1: Raw Data] X min/max: {X.min()}/{X.max()}")

        # 🚨 [수정 및 검증 2] 데이터 타입 명시적 변환
        X = X.astype(np.float64) 
        y = y.astype(np.int64) 

        # Subsample 및 분할 로직
        if n_samples > len(X): n_samples = len(X)
            
        X_sub, _, y_sub, _ = train_test_split(X, y, train_size=n_samples, random_state=random_state, stratify=y)
        
        X_train_sub, X_val_sub, Y_train_sub, Y_val_sub = train_test_split(
            X_sub, y_sub, test_size=0.3, random_state=random_state, stratify=y_sub
        )

        # 🚨 [검증 3] 결측치 확인 (스케일링 전)
        if np.isnan(X_train_sub).any() or np.isinf(X_train_sub).any():
            print("  [CHECK 3: NaN/Inf Alert] Found NaN/Inf values before scaling.")
            
        # 스케일링
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_sub)
        X_val_scaled = scaler.transform(X_val_sub)
        
        # 🚨 [검증 4] 스케일링 후 값 확인 (평균 0, 표준편차 1 근처여야 함)
        print(f"  [CHECK 4: Scaled Data] X_train mean/std: {np.mean(X_train_scaled):.4f} / {np.std(X_train_scaled):.4f}")


        X_TRAIN, Y_TRAIN = X_train_scaled, Y_train_sub
        X_VAL, Y_VAL = X_val_scaled, Y_val_sub
        
        print(f"Data Loaded: Train={len(X_TRAIN)} samples, Validation={len(X_VAL)} samples.")
        return X_TRAIN, Y_TRAIN, X_VAL, Y_VAL

    except Exception as e:
        print(f"Error during Fashion MNIST processing: {e}. Cannot proceed with HPO.")
        # HPO 진행 불가 시 시스템 종료
        sys.exit(1)


def rf_hpo_fitness_nd(point: Tuple[int, int, int]) -> float:
    """
    Random Forest HPO 목표 함수: (n_estimators, max_depth, max_features_VALUE)를 입력받아
    Fitness (1 - Accuracy)를 반환합니다.
    """
    
    # 1. 파라미터 매핑 및 제약 조건 적용
    n_estimators = max(50, min(200, int(point[0]))) # [50, 200]
    max_depth = max(10, min(30, int(point[1])))     # [10, 30]
    
    # 🚨 [수정된 부분]: x2를 정수값(50) 대신 비율(0.05 ~ 1.0)로 변환하여 사용합니다.
    # HPO framework는 정수 50을 전달하지만, ML 모델에는 0.05로 전달
    # 이 비율 파라미터가 Local Optima를 유도하기 좋습니다.
    # 예: point[2]=50 -> ratio=0.05 + 0.001*50 = 0.1
    
    # X_train을 로드하여 실제 특징 개수(784)를 얻습니다.
    X_train, Y_train, X_val, Y_val = load_fashion_mnist_for_hpo()
    
    # x2는 이제 비율(ratio)의 정수 인덱스처럼 사용됩니다.
    # x2의 탐색 범위 [30, 70]을 [0.05, 0.75] 비율로 매핑합니다.
    # 50은 중간 지점인 0.40에 해당되도록 설계
    feature_ratio_raw = int(point[2]) 
    
    # (30 -> 0.05) to (70 -> 0.75) mapping
    # Simple linear mapping: ratio = 0.05 + 0.01 * (x2 - 30)
    # (100, 10, 50) 시작점 가정: x2=50 -> ratio=0.05 + 0.01 * 20 = 0.25 
    
    # ratio_value = max(0.01, min(1.0, 0.05 + 0.01 * (feature_ratio_raw - 30))) # 30에서 시작했다고 가정하고 조정
    
    # HPO가 1부터 200까지 정수를 탐색하므로, x2 값을 1~200으로 보고 0.05~1.0 비율로 매핑합니다.
    # Start Point 50 -> 0.25 (적절한 값)
    max_features_ratio = max(0.05, min(1.0, feature_ratio_raw / 200.0))
    
    # 2. Model Training & Evaluation
    try:
        model = RandomForestClassifier(
            n_estimators=n_estimators, 
            max_depth=max_depth,
            max_features=max_features_ratio, # 🚨 float 비율을 전달
            random_state=42, 
            n_jobs=-1
        )
        model.fit(X_train, Y_train)
        
        y_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        fitness = 1.0 - accuracy
        
    except Exception as e:
        fitness = 1.0 

    return fitness

# ==============================================================================
# 4. MAIN EXECUTION (START POINT 및 HPO 설정 조정)
# ==============================================================================
# D=3 차원을 고려하여 START_POINT의 x2 값을 1~200 범위 내에서 설정합니다.
DIM = 3 
START_POINT_HPO = (50, 15, 50) # N=50, D=15, F_ratio=50/200=0.25 (안전한 성능 보장)
THRESHOLD = 0.00001
MAX_ITERATIONS = 10 
BASIN_MAX_SEARCH = 50

print("\n" + "="*80)
print("🎯 Random Forest HPO Comparison (D=3) - 학습 능력 복원")
print(f"   Parameters: (n_estimators, max_depth, max_features)")
print(f"   Start Point: {START_POINT_HPO}")
print("="*80)

# --- 1. Run with Compression HC ---
time_start_comp = time.time()
traj_comp = hill_climb_with_compression_nd(
    fitness_func_nd=rf_hpo_fitness_nd,
    start_point=START_POINT_HPO,
    dim=DIM,
    max_iterations=10, # Iteration 횟수 5 -> 10으로 늘려 Compression 기회 부여
    basin_max_search=20,
    global_min_threshold=THRESHOLD
)
time_end_comp = time.time()

# --- 2. Run Baseline HC ---
time_start_base = time.time()
traj_baseline = hill_climb_simple_nd(
    fitness_func_nd=rf_hpo_fitness_nd,
    start_point=START_POINT_HPO,
    dim=DIM,
    max_steps=500
)
time_end_base = time.time()

# ==============================================================================
# 5. FINAL SUMMARY
# ==============================================================================

def print_final_summary(traj, time_taken, label):
    if not traj:
        print(f"  {label:<30}: FAILED (No steps)")
        return 0.0
    
    final_f = traj[-1][1]
    final_p = traj[-1][0]
    accuracy = 1.0 - final_f
    steps = len(traj)
    
    print(f"  {label:<30}:")
    print(f"    - Final Params (N, D): {str(final_p):<25}")
    print(f"    - Final Fitness (Min): {final_f:<25.6f}")
    print(f"    - Final Accuracy:      {accuracy:<25.6f}")
    print(f"    - Total Steps:         {steps:<25}")
    print(f"    - Time Taken (s):      {time_taken:<25.4f}")
    
    return accuracy

print("\n" + "=="*40)
print("        FINAL RANDOM FOREST HPO COMPARISON")
print("=="*40)

acc_comp = print_final_summary(traj_comp, time_end_comp - time_start_comp, "Compression Hill Climbing")
acc_base = print_final_summary(traj_baseline, time_end_base - time_start_base, "Baseline Hill Climbing")

print("\n--- HPO Efficacy ---")
if acc_comp > acc_base + 1e-4:
    print(f"🎉 **Compression HC Succeeded!** (Accuracy improvement: {acc_comp - acc_base:.4f})")
else:
    print("⚠️ Baseline and Compression HC performed similarly.")


🎯 Random Forest HPO Comparison (D=3) - 학습 능력 복원
   Parameters: (n_estimators, max_depth, max_features)
   Start Point: (50, 15, 50)
Loading Fashion MNIST (Subsample for increased difficulty)...
  [CHECK 1: Raw Data] X Shape: (70000, 784), Y Type: int64
  [CHECK 1: Raw Data] X min/max: 0.0/255.0
  [CHECK 4: Scaled Data] X_train mean/std: -0.0000 / 1.0000
Data Loaded: Train=3500 samples, Validation=1500 samples.

🚀 3D COMPRESSION climb start at (50, 15, 50), f=1.0000

🔄 Iteration 1/10
  📍 Climbed 0 steps, now at (50, 15, 50), f=1

⚠️ STUCK at local minimum (50, 15, 50), f=1
  🔍 Detecting basins along all 3 dimensions...
    [Dim 0] (1D Detect @ x=50, f=1.0000) -> Basin: [30, 70] (len=41)
    [Dim 1] (1D Detect @ x=15, f=1.0000) -> Basin: [-5, 35] (len=41)
    [Dim 2] (1D Detect @ x=50, f=1.0000) -> Basin: [30, 70] (len=41)

  ⚠️ Restart candidates didn't improve fitness. Stopping.

🏁 FINAL 3D COMPRESSION RESULTS 🏁
  Final position: (50, 15, 50), Final fitness: 1, Total steps: 1

🚀 3D BA